<a href="https://colab.research.google.com/github/thulieblack/deeplearnings/blob/main/DisasterTweetsWithULMFiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 18.6MB/s 
     |████████████████████████████████| 1.2MB 55.5MB/s 
     |████████████████████████████████| 194kB 54.9MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 12.8MB 219kB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


#  What am I predicting?
You are predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.

# What Files Are Being Used?
train.csv - the training set
test.csv - the test set
sample_submission.csv - a sample submission file in the correct format

# What Columns Are Contained In The Dataset?
id - a unique identifier for each tweet
text - the text of the tweet
location - the location the tweet was sent from (may be blank)
keyword - a particular keyword from the tweet (may be blank)
target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

# What should I expect the data format to be?
Each sample in the train and test set has the following information:

The text of a tweet
A keyword from that tweet (although this may be blank!)
The location the tweet was sent from (may also be blank)

In [ ]:
from fastai.text.all import *
import pandas as pd

In [ ]:

from google.colab import files
uploaded = files.upload()

Saving test2.csv to test2.csv


In [ ]:

from google.colab import files
uploaded = files.upload()

Saving train2.csv to train2.csv


In [ ]:
df = pd.read_csv("train2.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [ ]:
df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d,1


In [ ]:
len(df)

7613

In [ ]:
test = pd.read_csv('test2.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
len(test)

3263

In [ ]:
# creating a language model

dls_lm = TextDataLoaders.from_df(df, text_col='text', is_lm=True, valid_pct=0.2)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls_lm.show_batch(max_n=3)

,text,text_
0,xxbos i entered to # win the xxup entire set of xxunk xxmaj lip xxmaj xxunk via xxunk . - xxmaj go enter ! # xxunk http : / / t.co / xxunk xxbos xxmaj to fight bioterrorism sir . xxbos xxmaj emergency xxmaj response and xxmaj hazardous xxmaj chemical xxmaj management : xxmaj xxunk and xxmaj xxunk http : / / t.co / xxunk http : / / t.co / xxunk,i entered to # win the xxup entire set of xxunk xxmaj lip xxmaj xxunk via xxunk . - xxmaj go enter ! # xxunk http : / / t.co / xxunk xxbos xxmaj to fight bioterrorism sir . xxbos xxmaj emergency xxmaj response and xxmaj hazardous xxmaj chemical xxmaj management : xxmaj xxunk and xxmaj xxunk http : / / t.co / xxunk http : / / t.co / xxunk xxbos
1,xxmaj on xxmaj all xxmaj the xxmaj corners xxmaj of xxmaj the xxmaj planet [ on xxmaj all xxmaj levels ] http : / / t.co / xxup xxunk xxbos xxmaj emergency xxmaj xxunk http : / / t.co / xxunk xxunk http : / / t.co / xxunk xxunk http : / / t.co / xxunk xxbos xxmaj xxunk xxmaj xxunk goes on the beat with xxmaj xxunk xxmaj police http,on xxmaj all xxmaj the xxmaj corners xxmaj of xxmaj the xxmaj planet [ on xxmaj all xxmaj levels ] http : / / t.co / xxup xxunk xxbos xxmaj emergency xxmaj xxunk http : / / t.co / xxunk xxunk http : / / t.co / xxunk xxunk http : / / t.co / xxunk xxbos xxmaj xxunk xxmaj xxunk goes on the beat with xxmaj xxunk xxmaj police http :
2,xxmaj xxunk [ big xxup band xxup mode ] for old - xxunk obliteration ! xxbos xxmaj railway xxmaj minister xxmaj prabhu calls xxup mp derailment a natural calamity http : / / t.co / xxunk xxbos xxmaj xxunk like xxunk salt in the wounds dad .. xxrep 6 ? http : / / t.co / xxunk xxbos xxmaj accident on xxup xxunk w # xxunk . xxmaj traffic moving 8 m,xxunk [ big xxup band xxup mode ] for old - xxunk obliteration ! xxbos xxmaj railway xxmaj minister xxmaj prabhu calls xxup mp derailment a natural calamity http : / / t.co / xxunk xxbos xxmaj xxunk like xxunk salt in the wounds dad .. xxrep 6 ? http : / / t.co / xxunk xxbos xxmaj accident on xxup xxunk w # xxunk . xxmaj traffic moving 8 m slower


In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.623936,3.845527,0.372879,46.783340,00:10


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.955050,3.630280,0.389876,37.723396,07:51
1,3.761971,3.408724,0.416661,30.226639,07:53
2,3.557854,3.300635,0.426592,27.129864,07:52
3,3.372389,3.161328,0.447300,23.601915,07:51
4,3.208343,3.107799,0.455095,22.371756,07:51
5,3.068883,3.073793,0.460502,21.623764,07:53
6,2.953365,3.055036,0.464264,21.221951,07:49
7,2.852714,3.043555,0.467057,20.979698,07:47
8,2.792358,3.043119,0.466699,20.970558,07:45
9,2.752334,3.043160,0.466546,20.971416,07:44


In [ ]:
learn.save_encoder('tuned')

In [ ]:
TEXT = "I know "
N_WORDS = 40
N_SENTENCES = 2

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i know what shit happens when you make me a crush RT Strategy : Reddit 's new content policy goes into effect many horrible subreddits banned or quarantined http : / / t.co / hijacking http : /
i know a fire truck is in the middle of my car . It 's going to explode . Not expected to be able to see the U.S . The Obama Declares Disaster for


In [ ]:
# text classifier

dls_clas = TextDataLoaders.from_df(df, text_col="text", label_col="target", valid_pct=0.2, text_vocab=dls_lm.vocab)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,xxbos _ \n▁ xxrep 5 ? xxup retweet \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup follow xxup all xxup who xxup rt \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup xxunk \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup gain xxup with \n▁ xxrep 7 ? \n▁ xxrep 5 ? xxup follow ? xxunk # xxup xxunk \n▁ # xxup ty,0
1,xxbos xxup info xxup s. xxup wnd : xxunk / 6 . xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup inst xxup apch . xxup rwy 05 . xxup curfew xxup in xxup oper xxup until 2030 xxup z. xxup taxiways xxup foxtrot 5 & & xxup foxtrot 6 xxup navbl . xxup tmp : 10 .,0
2,xxbos xxmaj truth … \n https : / / t.co / xxunk \n▁ # xxmaj news \n▁ # xxup bbc \n▁ # xxup cnn \n▁ # xxmaj islam \n▁ # xxmaj truth \n▁ # god \n▁ # xxup isis \n▁ # terrorism \n▁ # xxmaj quran \n▁ # xxmaj lies http : / / t.co / xxunk,1


In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn = learn.load_encoder('tuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.533363,0.458680,0.787122,00:09


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.511794,0.443534,0.789093,00:09


In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.481639,0.437549,0.798949,00:10


In [ ]:
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.405118,0.443128,0.808147,00:10
1,0.386749,0.441426,0.810775,00:10


In [ ]:
learn.show_results(max_n=5)

,text,category,category_
0,xxbos xxup info xxup u. xxup xxunk : xxup xxunk xxup xxunk . xxup exp xxup inst xxup apch . xxup rwy 05 . xxup curfew xxup in xxup oper xxup until 2030 xxup z. xxup taxiways xxup foxtrot 5 & & xxup foxtrot 6 xxup navbl . xxup tmp : 10 . xxup wnd : xxunk / 6 .,0,0
1,xxbos xxup info xxup r. xxup curfew xxup in xxup oper xxup until 2030 xxup z. xxup taxiways xxup foxtrot 5 & & xxup foxtrot 6 xxup navbl . xxup wnd : xxunk / 5 . xxup exp xxup inst xxup apch . xxup rwy 05 . xxup xxunk . xxup tmp : 10 . xxup xxunk : xxunk .,0,0
2,xxbos xxmaj truth … \n https : / / t.co / xxunk \n▁ # xxmaj news \n▁ # xxup bbc \n▁ # xxup cnn \n▁ # xxmaj islam \n▁ # xxmaj truth \n▁ # god \n▁ # xxup isis \n▁ # terrorism \n▁ # xxmaj quran \n▁ # xxmaj lies http : / / t.co / xxunk,0,1
3,xxbos xxmaj truth … \n https : / / t.co / xxunk \n▁ # xxmaj news \n▁ # xxup bbc \n▁ # xxup cnn \n▁ # xxmaj islam \n▁ # xxmaj truth \n▁ # god \n▁ # xxup isis \n▁ # terrorism \n▁ # xxmaj quran \n▁ # xxmaj lies http : / / t.co / xxunk,0,1
4,xxbos xxmaj truth … \n https : / / t.co / xxunk \n▁ # xxmaj news \n▁ # xxup bbc \n▁ # xxup cnn \n▁ # xxmaj islam \n▁ # xxmaj truth \n▁ # god \n▁ # xxup isis \n▁ # terrorism \n▁ # xxmaj quran \n▁ # xxmaj lies http : / / t.co / xxunk,1,1


In [ ]:
test.iloc[:10,3]

0                                                                  Just happened a terrible car crash
1                                    Heard about #earthquake is different cities, stay safe everyone.
2    there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
3                                                            Apocalypse lighting. #Spokane #wildfires
4                                                       Typhoon Soudelor kills 28 in China and Taiwan
5                                                                  We're shaking...It's an earthquake
6                            They'd probably still show more life than Arsenal did yesterday, eh? EH?
7                                                                                   Hey! How are you?
8                                                                                    What a nice hat?
9                                                                                 

In [ ]:
id_ , target_  = [], [] 
for i, text in zip(test.iloc[:,0], test.iloc[:,3]):
    id_.append(i)
    target_.append(int(learn.predict(text)[0]))
id_, target_

([0,
  2,
  3,
  9,
  11,
  12,
  21,
  22,
  27,
  29,
  30,
  35,
  42,
  43,
  45,
  46,
  47,
  51,
  58,
  60,
  69,
  70,
  72,
  75,
  84,
  87,
  88,
  90,
  94,
  99,
  101,
  103,
  106,
  108,
  111,
  115,
  116,
  122,
  123,
  124,
  125,
  127,
  140,
  142,
  147,
  148,
  150,
  152,
  154,
  155,
  166,
  167,
  169,
  177,
  179,
  181,
  186,
  188,
  189,
  192,
  200,
  202,
  206,
  207,
  214,
  217,
  223,
  224,
  227,
  228,
  230,
  233,
  234,
  236,
  239,
  250,
  255,
  257,
  259,
  275,
  278,
  282,
  284,
  286,
  288,
  292,
  295,
  300,
  304,
  305,
  306,
  308,
  311,
  317,
  319,
  323,
  324,
  325,
  326,
  333,
  339,
  342,
  343,
  350,
  351,
  357,
  359,
  362,
  366,
  367,
  369,
  373,
  374,
  376,
  377,
  378,
  379,
  382,
  385,
  387,
  388,
  391,
  392,
  395,
  399,
  400,
  403,
  405,
  408,
  411,
  414,
  416,
  417,
  422,
  425,
  428,
  430,
  431,
  433,
  434,
  439,
  441,
  449,
  458,
  460,
  464,
  473,
  488

In [ ]:
submission = pd.DataFrame(list(zip(id_, target_)), 
               columns =['id', 'target']) 

In [ ]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [ ]:
submission.to_csv('submission.csv', index = False)